# Lab Assignment One: Exploring Table Data

Gabs DiLiegro, London Kasper, Carys LeKander

# 1. Business Understanding

Housing prices are a large topic of interest for many people around the world. A person's home and environment can greatly affect their life and mental state. Being able to predict housing costs could help buyers and sellers in the housing market as people looking to buy a house could use this model to see what factors drive prices up and be able to stay in their price range and sellers could get an accurate estimate of how much their home is worth. 

Our dataset is centered around housing in neighborhoods around Melbourne, Australia. This data was collected from public records posted on Domain.com.au, and the prediction task associated with this dataset is the price of the housing. This dataset has 13580 entries with 21 attributes for each observation. We found this dataset at kaggle.com. 

With our visualizations and analysis, we are able to show the most important factors in determining the price of a house and eventually will be able to predict the price based on these factors. 

We believe that the accuracy of our model would need to be a high percentage to be accepted by users because there are many algorithms about predicting housing prices that currently exist with high accuracy. When looking online, we found multiple tools built to forecast prices of homes (ex. VeroFORECAST) and research papers about machine learning for predicitng housing costs (ex. one we found claims 86% accuracy). 

Dataset: https://www.kaggle.com/datasets/dansbecker/melbourne-housing-snapshot

******************************************************

VeroFORCAST: https://www.veros.com/solutions/home-price-trends-and-forecast/veroforecast

Reasearch paper example: https://www.ijert.org/comparison-of-machine-learning-algorithms-for-house-price-prediction-using-real-time-data


# 2. Data Understanding

In [9]:
import pandas as pd
import numpy as np

df = pd.read_csv('melb_data.csv')

# removing irrelevant columns 
df = df.drop(['Address', 'SellerG', 'Date', 'Propertycount'], axis=1)

df.head()

,Suburb,Rooms,Type,Price,Method,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname
0,Abbotsford,2,h,1480000.0,S,2.5,3067.0,2.0,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan
1,Abbotsford,2,h,1035000.0,S,2.5,3067.0,2.0,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan
2,Abbotsford,3,h,1465000.0,SP,2.5,3067.0,3.0,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan
3,Abbotsford,3,h,850000.0,PI,2.5,3067.0,3.0,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan
4,Abbotsford,4,h,1600000.0,VB,2.5,3067.0,3.0,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan


In [6]:
#find data types and missing data
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13580 entries, 0 to 13579
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Suburb         13580 non-null  object 
 1   Address        13580 non-null  object 
 2   Rooms          13580 non-null  int64  
 3   Type           13580 non-null  object 
 4   Price          13580 non-null  float64
 5   Method         13580 non-null  object 
 6   SellerG        13580 non-null  object 
 7   Date           13580 non-null  object 
 8   Distance       13580 non-null  float64
 9   Postcode       13580 non-null  float64
 10  Bedroom2       13580 non-null  float64
 11  Bathroom       13580 non-null  float64
 12  Car            13518 non-null  float64
 13  Landsize       13580 non-null  float64
 14  BuildingArea   7130 non-null   float64
 15  YearBuilt      8205 non-null   float64
 16  CouncilArea    12211 non-null  object 
 17  Lattitude      13580 non-null  float64
 18  Longti

## Data Dictionary 
- Suburb: the name of the suburb that each property is in. (String object)
- Rooms: the total number of rooms for each property. (int64)
- Type: the type of property. (String object: br = bedrooms; h = house/cottage/villa/seni/terrace; u = unit, duplex; t = townhouse; dev site = development site; ores = other residential)
- Price: listing price in Australian dollars (float64)
- Method: way the property was listed
- Distance: the distance from the property to the Melbourne central business district AKA CBD (float64)
- Postcode: zipcode the property falls within (float64) 
- Bedroom2: the number of bedrooms (float64: scraped from a different source)
- Bathroom: the number of bathrooms (float64)
- Car: the number of parking spots (float64)
- Landsize: the size of the land in meters (float64)
- BuildingArea: area of the building in meters
- YearBuilt: year the house was built (float64)
- CouncilArea: the governing council for the area (String object)
- Lattitude: lattitude of property (float64)
- Longtitude: longtitude of property (float64)
- Regionname: general region of the property (String object)

Columns removed:
- Address: the address of the property (String object)
- SellerG: name of the real estate agent listing the property (String object)
- Date: sale date in mm/dd/yyyy (float64)
- Propertycount: number of properties in the same suburb (float64)

## Data Quality Check
Many of our attributes already have null values as shown with df.info(), but there are likely other values that are either duplicated or incomplete. 

We believe that the BuildingArea and YearBuilt information could be useful in our model, but our dataset is missing a significant portion of the two so we decided to remove them. 

# 3. Data Visualization

# 4. Exceptional Work